## Задание
Что делать?

1.Датасет ml-latest

2.Вспомнить подходы, которые мы разбирали

3.Выбрать понравившийся подход к гибридным системам

4.Написать свою

Я выбрал метод поиска похожих фильмов.
Моя рекомендательная система работает по алгоритму:
1) создаю список жанров наиболее популярных у пользователя;
2) из дата фрейма выбираю фильмы которые пользователь не смотрел и оставляю в нём только фильмы с оценкой 5;
3) из этого датафрейма выбираю фильмы которые имеют максимальное совпадение по жанрам пользователя;
4) по оставшимся фильмам подбираю похожие из фильмов, которые он не смотрел.

PS. За основу взял материал лекции.

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.spatial.distance import cosine, euclidean, cityblock

In [2]:
df_ratings = pd.read_csv('ratings.csv')
df_movies = pd.read_csv('movies.csv')

In [3]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
df_movies.dropna(inplace = True)
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df = pd.merge(df_ratings, df_movies, on='movieId')
del df['timestamp']
df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


### Формируем векторное описание для фильма

In [6]:
df['userId'].describe()

count    100836.000000
mean        326.127564
std         182.618491
min           1.000000
25%         177.000000
50%         325.000000
75%         477.000000
max         610.000000
Name: userId, dtype: float64

In [7]:
MIN_USER_ID = df['userId'].min()
MAX_USER_ID = df['userId'].max()

In [8]:
movie_names = df['title'].unique()
movie_names = movie_names.tolist()
#movie_names

In [9]:
movie_to_vector = {}
for movie in tqdm_notebook(movie_names):
    movie_to_vector[movie] = np.zeros((MAX_USER_ID,))
    for r in df[df['title'] == movie].iterrows():
        movie_to_vector[movie][r[1]['userId'] - MIN_USER_ID] = r[1]['rating']

In [10]:
# функция ищет похожие фильмы.
def find_similar(movie, dist_func, top=10):
    distances = {}
    target_movie = movie_to_vector[movie]
    for m in movie_names:
        distances[m] = dist_func(target_movie, movie_to_vector[m])
        
    distances_with_idx = [(i, distances[m]) for i, m in enumerate(movie_names)]
    distances_with_idx = sorted(distances_with_idx, key=lambda t: t[1], reverse=False)
    distances_with_idx = distances_with_idx[:top]
    
    return [(movie_names[i], d) for i, d in distances_with_idx]

In [11]:
#Функция принимает на вход индекс юзера.
#возвращает датафрейм предпочтений пользователя отсортировынный по убыванию
#и индексы просмотренных им фильмов

def user_genre (user):

    user_movie = df.loc[df['userId'] == user]

    # Нахожу жанры всех фильмов пользователя
    user_genre_set = set()

    for s in user_movie['genres'].str.split('|').values:
        user_genre_set = user_genre_set.union(set(s))
    user_genre = list(user_genre_set)


    # Cчитаю количество вхождений жанров в фильмы пользователя
    user_genre_dict = {}

    for g in user_genre:
        t = 0
        for i in user_movie.index:
            if g in user_movie.genres[i].split('|'):
                t+=1
        user_genre_dict.update({g : t})
    user_df = pd.DataFrame(data = user_genre_dict.values(), index = user_genre_dict.keys())

    #сортирую по возврастанию
    user_df.sort_values(by = 0, ascending = False, inplace = True) 
    return user_df

#### В блоке ниже вводим userId для рекомендаций

In [12]:
user = 1
user_genre_list = user_genre(user)
print(user_genre_list[:3], '\n', 'Длина списка жанров:', len(user_genre_list))

            0
Action     90
Adventure  85
Comedy     83 
 Длина списка жанров: 17


In [13]:
#создаю датафрейм исключая из основного датафрейма фильмы, которые смотрел юзер и оставляя фильмы ретингом < 4.5   

df_out_user = df[(df.userId != user) & (df.rating > 4.5)]
df_out_user.drop_duplicates(subset = 'title', keep = 'first', inplace = True) 

df_out_user.head()

D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,userId,movieId,rating,title,genres
9,31,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
216,6,3,5.0,Grumpier Old Men (1995),Comedy|Romance
269,11,6,5.0,Heat (1995),Action|Crime|Thriller
373,13,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
576,8,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [14]:
# код считает количество совпадений жанров фильма ['count_fech'] с фильмами из user_genre_list.
# индексы фильмов помещаю в counter2.index

counter2 = pd.DataFrame(data = 0, index = df_out_user.index, columns = ['count_fech', 'title'])

for ind in df_out_user.index:
    for gen in user_genre_list.index:
        if gen in df_out_user['genres'][ind].split('|'):
            counter2['count_fech'][ind] +=1
            counter2['title'][ind] = df_out_user['title'][ind]

counter2.head()

D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,count_fech,title
9,5,Toy Story (1995)
216,2,Grumpier Old Men (1995)
269,3,Heat (1995)
373,2,Seven (a.k.a. Se7en) (1995)
576,3,"Usual Suspects, The (1995)"


# Рекомендации для пользователя

In [15]:

for movie in counter2.title[:3]:
    print()
    print(find_similar(movie, cityblock))


[('Toy Story (1995)', 0.0), ('Toy Story 2 (1999)', 608.5), ("Bug's Life, A (1998)", 698.5), ('Groundhog Day (1993)', 714.0), ('Nutty Professor, The (1996)', 714.0), ('Willy Wonka & the Chocolate Factory (1971)', 718.0), ('Mission: Impossible (1996)', 722.0), ('Babe (1995)', 722.5), ('Monsters, Inc. (2001)', 725.0), ('Toy Story 3 (2010)', 728.0)]

[('Grumpier Old Men (1995)', 0.0), ('Juror, The (1996)', 155.5), ('Kazaam (1996)', 155.5), ("Joe's Apartment (1996)", 155.5), ('Seventh Sign, The (1988)', 155.5), ('Flipper (1996)', 156.0), ('Tales from the Crypt Presents: Bordello of Blood (1996)', 156.0), ('Son in Law (1993)', 156.5), ('Dead Man on Campus (1998)', 156.5), ('Grumpy Old Men (1993)', 157.0)]

[('Heat (1995)', 0.0), ('Rumble in the Bronx (Hont faan kui) (1995)', 354.5), ('Strange Days (1995)', 357.5), ('Executive Decision (1996)', 364.0), ('First Strike (Police Story 4: First Strike) (Ging chaat goo si 4: Ji gaan daan yam mo) (1996)', 371.0), ('City Hall (1996)', 371.5), ('Hard